In [14]:
from ophyd.utils.epics_pvs import fmt_time

from ophyd.signal import (EpicsSignal, EpicsSignalRO)
from ophyd.utils import DisconnectedError
from ophyd.utils.epics_pvs import (raise_if_disconnected, AlarmSeverity)
from ophyd.positioner import PositionerBase
from ophyd.device import (Device, Component as Cpt)
from ophyd.status import wait as status_wait
from enum import Enum

class HomeEnum(str, Enum):
    forward = "forward"
    reverse = "reverse"


class EpicsGapMotor(Device, PositionerBase):
    '''An EPICS motor record, wrapped in a :class:`Positioner`

    Keyword arguments are passed through to the base class, Positioner

    Parameters
    ----------
    prefix : str
        The record to use
    read_attrs : sequence of attribute names
        The signals to be read during data acquisition (i.e., in read() and
        describe() calls)
    name : str, optional
        The name of the device
    parent : instance or None
        The instance of the parent device, if applicable
    settle_time : float, optional
        The amount of time to wait after moves to report status completion
    timeout : float, optional
        The default timeout to use for motion requests, in seconds.
    '''
    # position
    user_readback = Cpt(EpicsSignalRO, '.RBV', kind='hinted')
    user_setpoint = Cpt(EpicsSignal, '-SP', limits=True)

    # calibration dial <-> user
    user_offset = Cpt(EpicsSignal, '.OFF', kind='config')
    user_offset_dir = Cpt(EpicsSignal, '.DIR', kind='config')
    offset_freeze_switch = Cpt(EpicsSignal, '.FOFF', kind='omitted')
    set_use_switch = Cpt(EpicsSignal, '.SET', kind='omitted')

    # configuration
    velocity = Cpt(EpicsSignal, '.VELO', kind='config')
    acceleration = Cpt(EpicsSignal, '.ACCL', kind='config')
    motor_egu = Cpt(EpicsSignal, '.EGU', kind='config')

    # motor status
    motor_is_moving = Cpt(EpicsSignalRO, '.MOVN', kind='omitted')
    motor_done_move = Cpt(EpicsSignalRO, '.DMOV', kind='omitted')
    high_limit_switch = Cpt(EpicsSignal, '.HLS', kind='omitted')
    low_limit_switch = Cpt(EpicsSignal, '.LLS', kind='omitted')
    direction_of_travel = Cpt(EpicsSignal, '.TDIR', kind='omitted')

    # commands
    motor_stop = Cpt(EpicsSignal, '.STOP', kind='omitted')
    home_forward = Cpt(EpicsSignal, '.HOMF', kind='omitted')
    home_reverse = Cpt(EpicsSignal, '.HOMR', kind='omitted')

    # alarm information
    tolerated_alarm = AlarmSeverity.NO_ALARM

    def __init__(self, *args, **kwargs):

        super().__init__(*args, **kwargs)

        # Make the default alias for the user_readback the name of the
        # motor itself.
        self.user_readback.name = self.name

        self.motor_done_move.subscribe(self._move_changed)
        self.user_readback.subscribe(self._pos_changed)

    @property
    @raise_if_disconnected
    def precision(self):
        '''The precision of the readback PV, as reported by EPICS'''
        return self.user_readback.precision

    @property
    @raise_if_disconnected
    def egu(self):
        '''The engineering units (EGU) for a position'''
        return self.motor_egu.get()

    @property
    @raise_if_disconnected
    def limits(self):
        return self.user_setpoint.limits

    @property
    @raise_if_disconnected
    def moving(self):
        '''Whether or not the motor is moving

        Returns
        -------
        moving : bool
        '''
        return bool(self.motor_is_moving.get(use_monitor=False))

    @raise_if_disconnected
    def stop(self, *, success=False):
        self.motor_stop.put(1, wait=False)
        super().stop(success=success)

    @raise_if_disconnected
    def move(self, position, wait=True, **kwargs):
        '''Move to a specified position, optionally waiting for motion to
        complete.

        Parameters
        ----------
        position
            Position to move to
        moved_cb : callable
            Call this callback when movement has finished. This callback must
            accept one keyword argument: 'obj' which will be set to this
            positioner instance.
        timeout : float, optional
            Maximum time to wait for the motion. If None, the default timeout
            for this positioner is used.

        Returns
        -------
        status : MoveStatus

        Raises
        ------
        TimeoutError
            When motion takes longer than `timeout`
        ValueError
            On invalid positions
        RuntimeError
            If motion fails other than timing out
        '''
        self._started_moving = False

        status = super().move(position, **kwargs)
        self.user_setpoint.put(position, wait=False)
        try:
            if wait:
                status_wait(status)
        except KeyboardInterrupt:
            self.stop()
            raise

        return status

    @property
    @raise_if_disconnected
    def position(self):
        '''The current position of the motor in its engineering units

        Returns
        -------
        position : float
        '''
        return self._position

    @raise_if_disconnected
    def set_current_position(self, pos):
        '''Configure the motor user position to the given value

        Parameters
        ----------
        pos
           Position to set.

        '''
        self.set_use_switch.put(1, wait=True)
        self.user_setpoint.put(pos, wait=True)
        self.set_use_switch.put(0, wait=True)

    @raise_if_disconnected
    def home(self, direction, wait=True, **kwargs):
        '''Perform the default homing function in the desired direction

        Parameters
        ----------
        direction : HomeEnum
           Direction in which to perform the home search.
        '''
        direction = HomeEnum(direction)

        self._started_moving = False
        position = (self.low_limit+self.high_limit)/2
        status = super().move(position, **kwargs)

        if direction == HomeEnum.forward:
            self.home_forward.put(1, wait=False)
        else:
            self.home_reverse.put(1, wait=False)

        try:
            if wait:
                status_wait(status)
        except KeyboardInterrupt:
            self.stop()
            raise

        return status

    def check_value(self, pos):
        '''Check that the position is within the soft limits'''
        self.user_setpoint.check_value(pos)

    def _pos_changed(self, timestamp=None, value=None, **kwargs):
        '''Callback from EPICS, indicating a change in position'''
        self._set_position(value)

    def _move_changed(self, timestamp=None, value=None, sub_type=None,
                      **kwargs):
        '''Callback from EPICS, indicating that movement status has changed'''
        was_moving = self._moving
        self._moving = (value != 1)

        started = False
        if not self._started_moving:
            started = self._started_moving = (not was_moving and self._moving)

        self.log.debug('[ts=%s] %s moving: %s (value=%s)', fmt_time(timestamp),
                       self, self._moving, value)

        if started:
            self._run_subs(sub_type=self.SUB_START, timestamp=timestamp,
                           value=value, **kwargs)

        if was_moving and not self._moving:
            success = True
            # Check if we are moving towards the low limit switch
            if self.direction_of_travel.get() == 0:
                if self.low_limit_switch.get() == 1:
                    success = False
            # No, we are going to the high limit switch
            else:
                if self.high_limit_switch.get() == 1:
                    success = False

            # Check the severity of the alarm field after motion is complete.
            # If there is any alarm at all warn the user, and if the alarm is
            # greater than what is tolerated, mark the move as unsuccessful
            severity = self.user_readback.alarm_severity

            if severity != AlarmSeverity.NO_ALARM:
                status = self.user_readback.alarm_status
                if severity > self.tolerated_alarm:
                    self.log.error('Motion failed: %s is in an alarm state '
                                   'status=%s severity=%s',
                                   self.name, status, severity)
                    success = False
                else:
                    self.log.warning('Motor %s raised an alarm during motion '
                                     'status=%s severity %s',
                                     self.name, status, severity)
            self._done_moving(success=success, timestamp=timestamp,
                              value=value)

    @property
    def report(self):
        try:
            rep = super().report
        except DisconnectedError:
            # TODO there might be more in this that gets lost
            rep = {'position': 'disconnected'}
        rep['pv'] = self.user_readback.pvname
        return rep




In [15]:
gap = EpicsGapMotor("SR:C16-ID:G1{IVU23:1-Ax:Gap}-Mtr", name='gap')

In [16]:
gap.position

20000.041

In [4]:
gap.signal_names

/opt/conda_envs/collection-2018-3.0/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: 'signal_names' has been renamed 'component_names' for clarity because it may include a mixture of Signals and Devices -- any Components. This alias may be removed in a future release of ophyd.
  if __name__ == '__main__':


('user_readback',
 'user_setpoint',
 'user_offset',
 'user_offset_dir',
 'offset_freeze_switch',
 'set_use_switch',
 'velocity',
 'acceleration',
 'motor_egu',
 'motor_is_moving',
 'motor_done_move',
 'high_limit_switch',
 'low_limit_switch',
 'direction_of_travel',
 'motor_stop',
 'home_forward',
 'home_reverse')

In [ ]:
gap.user_setpoint, gap.user_readback, gap.motor_stop

In [18]:
%mov gap 18000

gap:   0%|                            | 0.01/2000.04 [00:00<1:32:39,  2.78s/um]
gap:   7%|█▉                         | 141.72/2000.04 [00:00<00:07, 265.02um/s]
gap:  10%|██▋                        | 196.19/2000.04 [00:00<00:05, 300.71um/s]
gap:  12%|███▎                       | 249.35/2000.04 [00:00<00:05, 328.34um/s]
gap:  18%|████▊                      | 357.28/2000.04 [00:00<00:04, 363.74um/s]
gap:  21%|█████▋                     | 417.44/2000.04 [00:01<00:04, 379.80um/s]
gap:  24%|██████▍                    | 474.77/2000.04 [00:01<00:03, 393.11um/s]
gap:  26%|███████▏                   | 528.61/2000.04 [00:01<00:03, 402.48um/s]
gap:  29%|███████▊                   | 583.24/2000.04 [00:01<00:03, 407.31um/s]
gap:  32%|████████▋                  | 640.98/2000.04 [00:01<00:03, 416.40um/s]
gap:  35%|█████████▍                 | 696.05/2000.04 [00:01<00:03, 422.98um/s]
gap:  38%|██████████▏                | 751.09/2000.04 [00:01<00:02, 428.46um/s]
gap:  43%|███████████▌               | 8

In [21]:
login('test', 'test', 'test')

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /GPFS/xf16id/exp_path/test/test/log-test.2018Dec04_10:23:15
Mode           : backup
Output logging : True
Raw input log  : True
Timestamping   : True
State          : active


In [23]:
RE(ascan([det], gap, 5800, 6200, 41))

updating meta data ...Done.
Transient Scan ID: 2     Time: 2018/12/04 10:24:13
Persistent Unique Scan ID: '91392f78-a600-4c2a-aa01-afa67fccb204'
Scan ID: 2
Unique ID: 91392f78-a600-4c2a-aa01-afa67fccb204
#STARTDOC : 1543937053.6162643
#STARTDOC : Tue Dec  4 10:24:13 2018


/opt/conda_envs/collection-2018-3.0/lib/python3.6/site-packages/nslsii/__init__.py:262: UserWarning: This olog is giving errors. This will not be logged.Error:HTTPSConnectionPool(host='xf16idc-ca.cs.nsls2.local', port=9191): Read timed out. (read timeout=30)
  'Error:' + str(exc))


New stream: 'primary'
+-----------+------------+------------+
|   seq_num |       time |        gap |
+-----------+------------+------------+
|         1 | 10:24:40.2 |    5800.06 |
|         2 | 10:24:41.0 |    5809.99 |
|         3 | 10:24:42.0 |    5819.98 |
|         4 | 10:24:42.7 |    5830.02 |
|         5 | 10:24:43.5 |    5839.91 |
|         6 | 10:24:44.2 |    5850.03 |
|         7 | 10:24:45.0 |    5859.99 |
|         8 | 10:24:45.7 |    5869.98 |
|         9 | 10:24:46.6 |    5880.02 |
|        10 | 10:24:47.3 |    5889.96 |
|        11 | 10:29:20.0 |    5899.92 |
|        12 | 10:29:20.8 |    5910.06 |
|        13 | 10:29:21.6 |    5919.94 |
|        14 | 10:29:22.3 |    5930.05 |
|        15 | 10:30:12.7 |    5940.03 |
A 'deferred pause' has been requested. The RunEngine will pause at the next checkpoint. To pause immediately, hit Ctrl+C again in the next 10 seconds.
Deferred pause acknowledged. Continuing to checkpoint.
|        16 | 10:30:13.5 |    5949.94 |
Pausing...


RunEngineInterrupted: 
Your RunEngine is entering a paused state. These are your options for changing
the state of the RunEngine:

RE.resume()    Resume the plan.
RE.abort()     Perform cleanup, then kill plan. Mark exit_stats='aborted'.
RE.stop()      Perform cleanup, then kill plan. Mark exit_status='success'.
RE.halt()      Emergency Stop: Do not perform cleanup --- just stop.


In [24]:
RE.abort()

Aborting: running cleanup and marking exit_status as 'abort'...
+-----------+------------+------------+
generator scan ['91392f78'] (scan num: 2)



Scan ID: 2
Unique ID: 91392f78-a600-4c2a-aa01-afa67fccb204


['91392f78-a600-4c2a-aa01-afa67fccb204']